## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

In [49]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.sql.functions import count, when, col, mean, rand

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!</font>

In [50]:
spark = SparkSession.builder.appName("Housing").getOrCreate()

# Подготовка данных

In [51]:
data = spark.read.csv("/datasets/housing.csv", header=True, inferSchema=True)

Вывод типов данных колонок датасета

In [52]:
data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [53]:
# Проверка наличия пропущенных значений в данных
missing_values = data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).toPandas()
print(missing_values)

   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0          0         0                   0            0             207   

   population  households  median_income  median_house_value  ocean_proximity  
0           0           0              0                   0                0  


Заполнение пропущенных значений

In [54]:
data = data.fillna(1)

In [55]:
# повторная проверка наличия пропущенных значений в данных
missing_values = data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).toPandas()
print(missing_values)

   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0          0         0                   0            0               0   

   population  households  median_income  median_house_value  ocean_proximity  
0           0           0              0                   0                0  


In [57]:
data = data.orderBy(rand())

In [63]:
train_data, test_data = data.randomSplit([0.7, 0.3])

Построение модели пайплайном

исправил

In [64]:
# Определение шагов предобработки данных
indexer = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_proximity_index")
encoder = OneHotEncoder(inputCol="ocean_proximity_index", outputCol="ocean_proximity_encoded")

# Обучение моделей

In [65]:
# Создание ассемблеров для двух моделей
assembler1 = VectorAssembler(inputCols=["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms",
                                        "population", "households", "median_income", "ocean_proximity_encoded"],
                             outputCol="features")
assembler2 = VectorAssembler(inputCols=["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms",
                                        "population", "households", "median_income"],
                             outputCol="features")

Выполнение предсказания на тестовой выборке

In [66]:
# Создание моделей линейной регрессии
lr1 = LinearRegression(featuresCol="features", labelCol="median_house_value")
lr2 = LinearRegression(featuresCol="features", labelCol="median_house_value")

In [67]:
# Создание двух конвейеров для двух моделей
pipeline1 = Pipeline(stages=[indexer, encoder, assembler1, lr1])
pipeline2 = Pipeline(stages=[assembler2, lr2])

In [68]:
# Обучение моделей на обучающих данных
model1 = pipeline1.fit(train_data)
model2 = pipeline2.fit(train_data)

23/10/17 12:36:35 WARN Instrumentation: [6605afab] regParam is zero, which might cause numerical instability and overfitting.
23/10/17 12:36:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/10/17 12:36:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/10/17 12:36:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/10/17 12:36:36 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
23/10/17 12:36:38 WARN Instrumentation: [d513092b] regParam is zero, which might cause numerical instability and overfitting.


In [70]:
# Выполнение предсказания на тестовой выборке
predictions1 = model1.transform(test_data)
predictions2 = model2.transform(test_data)

# Анализ результатов

Вычисление метрик RMSE, MAE и R2

In [71]:
# Вычисление метрик RMSE, MAE и R2
evaluator = RegressionEvaluator(labelCol="median_house_value")
rmse1 = evaluator.evaluate(predictions1, {evaluator.metricName: "rmse"})
mae1 = evaluator.evaluate(predictions1, {evaluator.metricName: "mae"})
r2_1 = evaluator.evaluate(predictions1, {evaluator.metricName: "r2"})

rmse2 = evaluator.evaluate(predictions2, {evaluator.metricName: "rmse"})
mae2 = evaluator.evaluate(predictions2, {evaluator.metricName: "mae"})
r2_2 = evaluator.evaluate(predictions2, {evaluator.metricName: "r2"})


 Вывод результатов

In [72]:
# Вывод результатов
print("Модель с использованием всех данных:")
print(f"RMSE: {rmse1}")
print(f"MAE: {mae1}")
print(f"R2: {r2_1}")

print("Модель только с числовыми переменными:")
print(f"RMSE: {rmse2}")
print(f"MAE: {mae2}")
print(f"R2: {r2_2}")

Модель с использованием всех данных:
RMSE: 70209.90753271118
MAE: 50909.02060464449
R2: 0.6389181494753682
Модель только с числовыми переменными:
RMSE: 71328.46045645037
MAE: 52001.259313827926
R2: 0.6273213117961614


Исходя из предоставленных результатов, можно сделать следующие аналитические выводы:

Сравнение моделей:

"Модель с использованием всех данных" имеет ниже значение RMSE (70209.91) и MAE (50909.02), что говорит о более точных прогнозах стоимости домов в жилом массиве, по сравнению с "Модель только с числовыми переменными", у которой RMSE равен 71328.46 и MAE равен 52001.26.
Коэффициент детерминации R2 говорит о том, что около 63,9% дисперсии стоимости домов можно объяснить предсказанными переменными в "Модели с использованием всех данных" и около 62,7% в "Модели только с числовыми переменными". Чем ближе R2 к единице, тем лучше модель объясняет данные.
Оценка модели:

RMSE (корень из среднеквадратичной ошибки) измеряет среднюю ошибку модели и показывает разброс между фактическими значениями и предсказаниями модели. В данном случае, значения RMSE близки друг к другу для обеих моделей, но "Модель с использованием всех данных" демонстрирует немного лучшие результаты.
MAE (средняя абсолютная ошибка) также измеряет среднюю ошибку модели, но в абсолютных величинах, что позволяет легче интерпретировать ошибки. Аналогично RMSE, "Модель с использованием всех данных" демонстрирует немного лучшие результаты.
Коэффициент детерминации R2 используется для оценки объяснительной способности модели. Значение R2 близкое к 1 указывает на лучшую способность модели объяснить и предсказать зависимую переменную. В данном случае, "Модель с использованием всех данных" имеет немного более высокое значение R2, что говорит о ее лучшей способности объяснить стоимость домов.
